In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
import os
import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

In [2]:
path = os.getcwd()
print(path)

/workspace/code


In [3]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 20)

In [4]:
df_train = pd.read_csv('/workspace/data/train.csv')
df_test =pd.read_csv('/workspace/data/test.csv')

In [68]:
df_sub = pd.read_csv('/workspace/data/sample_submission.csv')

In [5]:
df_train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,...,0,0,21,0,0,0,0,0,0,Class_2
1,1,0,0,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_1
2,2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,13,2,0,Class_1
3,3,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,1,0,Class_4
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,Class_2


## 取り合えずLightGBMで学習

In [ ]:
#### とりあえず目的変数をラベルエンコーディング

In [5]:
#ターゲットをラベルエンコーディング
le = LabelEncoder()
df_train['target'] = le.fit_transform(df_train['target'].values)

In [6]:
# idを削除
df_train = df_train.drop(columns='id')
df_test = df_test.drop(columns='id')

In [47]:
df_test

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,0,0,0,0,0,4,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,2,0,7,7,...,3,0,1,0,0,0,1,0,2,1
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,6,0
3,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,6,9,14,3
4,0,0,0,0,0,0,1,0,4,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,1,5,0,...,0,1,0,0,0,3,0,0,3,0
49996,0,0,0,0,0,0,10,0,0,0,...,0,0,0,1,0,0,0,0,0,3
49997,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
49998,0,0,0,0,1,0,0,0,0,4,...,1,0,0,0,0,0,0,0,0,1


In [7]:
# 分布に大きな不均衡がある場合に用いるKFold．
# 分布(y=1)の比率を維持したままデータを訓練用とテスト用に分割する．
from sklearn.model_selection import StratifiedKFold

In [65]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# マルチクラス分類のため出力は４列の確率値になるため出力用のnumpy.arrayは100000×４列となる
oof_preds = np.zeros((df_train.shape[0],4))
sub_preds = np.zeros((df_test.shape[0],4))
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df_train.iloc[:,0:-1],df_train['target'])):
    X_train, y_train = df_train.iloc[train_idx,0:-1], df_train['target'].iloc[train_idx]
    X_valid, y_valid = df_train.iloc[valid_idx,0:-1], df_train['target'].iloc[valid_idx]
    
    clf = LGBMClassifier(
        n_estimators=2000,
        learning_rate=0.1,
        num_leaves=42,
        colsample_bytree=.8,
        subsample=.9,
        max_depth=15,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01,
        min_child_weight=2
    )
    
    clf.fit(X_train, y_train, 
            eval_set= [(X_train, y_train), (X_valid, y_valid)], 
            eval_metric='multi_logloss', verbose=250, early_stopping_rounds=150,
            
           )
    
    oof_preds[valid_idx] = clf.predict_proba(X_valid, num_iteration=clf.best_iteration_)[:, :]
    sub_preds += clf.predict_proba(df_test, num_iteration=clf.best_iteration_)[:, :] / folds.n_splits

Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[54]	training's multi_logloss: 1.0371	valid_1's multi_logloss: 1.09748
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[50]	training's multi_logloss: 1.04094	valid_1's multi_logloss: 1.09635
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[66]	training's multi_logloss: 1.02686	valid_1's multi_logloss: 1.09429
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[56]	training's multi_logloss: 1.03542	valid_1's multi_logloss: 1.09564
Training until validation scores don't improve for 150 rounds
Early stopping, best iteration is:
[64]	training's multi_logloss: 1.02875	valid_1's multi_logloss: 1.09489


In [74]:
oof_preds

array([[0.07352532, 0.45739472, 0.32169016, 0.1473898 ],
       [0.08852276, 0.54590745, 0.23437475, 0.13119504],
       [0.15577423, 0.42114408, 0.30970122, 0.11338047],
       ...,
       [0.09672309, 0.55802604, 0.22177747, 0.12347341],
       [0.07449304, 0.66401815, 0.13533196, 0.12615684],
       [0.08686241, 0.49926302, 0.27535674, 0.13851783]])

In [67]:
sub_preds[:,0]

array([0.08018661, 0.07038145, 0.08449084, ..., 0.08388303, 0.08369156,
       0.10843888])

In [71]:
submission = pd.DataFrame({'id': df_sub['id'],
                           'Class_1': sub_preds[:, 0],
                           'Class_2': sub_preds[:, 1],
                           'Class_3': sub_preds[:, 2],
                           'Class_4': sub_preds[:, 3],})

In [72]:
submission.to_csv('lightGBM.csv',index=False)

In [75]:
submission

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.080187,0.591489,0.201615,0.126709
1,100001,0.070381,0.644365,0.190690,0.094564
2,100002,0.084491,0.639185,0.180423,0.095901
3,100003,0.082368,0.569509,0.240811,0.107312
4,100004,0.075704,0.612805,0.214957,0.096534
...,...,...,...,...,...
49995,149995,0.095471,0.676105,0.151875,0.076550
49996,149996,0.080826,0.659209,0.141833,0.118132
49997,149997,0.083883,0.520544,0.217232,0.178341
49998,149998,0.083692,0.602746,0.163676,0.149886
